# Webscraping S9
## Dice
## DIA1, A5

In [1]:
# !pip install beautifulsoup4

In [1]:
# !pip install selenium undetected_chromedriver

In [16]:
import re 
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import numpy as np

from undetected_chromedriver import Chrome

## Projet:
Scrap the following:  
https://dice.fm/browse/paris/music/gig  
https://dice.fm/browse/paris/music/party  
https://dice.fm/browse/paris/music/dj  

On these pages are all the links to the events themselves. I need to first get all those links, and then access every link and scrap the event data.

## Part 1: Scrap links for all events

In [22]:
# Initialize the webdriver (in this example, using Chrome)
driver = webdriver.Chrome()

# Navigate to your website
# website_url = 'https://dice.fm/browse/paris/music/gig'
# website_url = 'https://dice.fm/browse/paris/music/party'
website_url = 'https://dice.fm/browse/paris/music/dj'
driver.get(website_url)

#### Click 'voir plus' button

In [23]:
# Define the button XPaths
accept_cookies_button_xpath = '//button[@class="ch2-btn ch2-allow-all-btn ch2-btn-primary ch2-btn-text-xxs"]'
load_more_button_xpath = '//div[@class="styles__LoadMoreRow-sc-f7f02326-1 ixkwan"]//button'

try:
    # Wait for the "Accept All Cookies" button to be clickable
    accept_cookies_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, accept_cookies_button_xpath))
    )

    # Click the "Accept All Cookies" button
    accept_cookies_button.click()

    while True:
        # Wait for the "Load More" button to be clickable
        load_more_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, load_more_button_xpath))
        )

        # Click the "Load More" button
        load_more_button.click()


except TimeoutException:
    print("No more items to load.")

No more items to load.


In [24]:
page_source = driver.page_source
soup = BeautifulSoup(page_source,'html.parser')

On récupère le lien de l'événement, le nom, la date, le lieu, le prix (link, title, date, venue, price)

In [25]:
# df_dice = pd.DataFrame([], columns=['title', 'date', 'venue', 'price', 'link'])

In [26]:
# Find all 'EventCard' blocks
event_blocks = soup.find_all('div', class_='EventCard__Event-sc-96055878-1 jCnRTS')
i=0 
    
# Iterate through each 'EventCard' block and extract information
for event_block in event_blocks:
    title_element = event_block.find('div', class_='styles__Title-sc-bcd2794e-6 fLvsjM')
    date_element = event_block.find('div', class_='styles__Date-sc-bcd2794e-8 cbhNxH')
    venue_element = event_block.find('div', class_='styles__Venue-sc-bcd2794e-7 kJglZg')
    price_element = event_block.find('div', class_='styles__Price-sc-bcd2794e-9 kzrFAz')
    link_element = event_block.find('a', class_='styles__EventCardLink-sc-bcd2794e-5 hCzsEw')

    # Check if the elements are found before accessing their text attributes
    if title_element:
        title = title_element.text.strip()
    else:
        title=np.nan
        
    if date_element:
        date = date_element.text.strip()
    else:
        date=np.nan
    
    if venue_element:
        venue = venue_element.text.strip()
    else:
        venue=np.nan
    
    if price_element: 
        price = price_element.text.strip()
    else:
        price=np.nan
    
    if link_element:
        link = "https://dice.fm" + link_element['href']
    else:
        link=np.nan

    df_dice.loc[len(df_dice)] = [title, date, venue, price, link]
    i += 1

In [27]:
df_dice.head()

,title,date,venue,price,link
0,Les Clopes + Scimia + Blank\\,jeu. 4 janv.,Supersonic,Gratuit,https://dice.fm/event/qp2ny-les-clopes-scimia-...
1,Plastic Palms + Clamn Dever + Panty Panty,jeu. 4 janv.,L'international,8 €,https://dice.fm/event/xym5a-plastic-palms-clam...
2,Simone's Carpet + Lavomatic,ven. 5 janv.,Supersonic,Gratuit,https://dice.fm/event/373kp-simones-carpet-lav...
3,The Wolf You Feed + Gargantura + Cerbera,ven. 5 janv.,L'international,7 €,https://dice.fm/event/v2wkl-the-wolf-you-feed-...
4,La nuit des chansons tristes,ven. 5 janv.,Anti Club du Cirque électrique,8 €,https://dice.fm/event/v2vvg-la-nuit-des-chanso...


In [28]:
df_dice.shape

(1754, 5)

In [32]:
df_dice.isnull().sum()

title    0
date     0
venue    0
price    5
link     0
dtype: int64

In [31]:
# df_dice.to_csv('df_dice_1stPART-WebScraping_S9.csv')

In [3]:
# df_dice=pd.read_csv(r'df_dice_1stPART-WebScraping_S9.csv')
# df_dice = df_dice.drop(columns=['Unnamed: 0'])

In [9]:
df_dice.head()

,title,date,venue,price,link
0,Les Clopes + Scimia + Blank\\,jeu. 4 janv.,Supersonic,Gratuit,https://dice.fm/event/qp2ny-les-clopes-scimia-...
1,Plastic Palms + Clamn Dever + Panty Panty,jeu. 4 janv.,L'international,8 €,https://dice.fm/event/xym5a-plastic-palms-clam...
2,Simone's Carpet + Lavomatic,ven. 5 janv.,Supersonic,Gratuit,https://dice.fm/event/373kp-simones-carpet-lav...
3,The Wolf You Feed + Gargantura + Cerbera,ven. 5 janv.,L'international,7 €,https://dice.fm/event/v2wkl-the-wolf-you-feed-...
4,La nuit des chansons tristes,ven. 5 janv.,Anti Club du Cirque électrique,8 €,https://dice.fm/event/v2vvg-la-nuit-des-chanso...


## Part 2: Scrap event data for every event whose link was previously collected and stored in df_dice dataframe
We get the date and time of the event, the artists, the address of the venue, and the link to the venue's page

### Test scraping d'une page d'event
A mettre ensuite dans une boucle pour faire tous les événements du df_dice

In [48]:
# Initialize the webdriver (in this example, using Chrome)
driver = webdriver.Chrome()

# Navigate to your website
website_url ='https://dice.fm/event/png2p-rock-around-the-clock-5th-jan-supersonic-paris-tickets' # 'https://dice.fm/event/ar9pv-blue-jay-pythies-dj-set-22nd-dec-le-sub-pigalle-paris-tickets'
driver.get(website_url)

In [49]:
page_source = driver.page_source
soup = BeautifulSoup(page_source,'html.parser')

In [51]:
date_and_time_element = soup.find('div', class_='EventDetailsTitle__Date-sc-dfc77feb-2 bITArK')
location_element = soup.find('div', class_='EventDetailsVenue__Address-sc-456e9ee2-5 bGzzGO').find('span')


    
if date_and_time_element:
    date_and_time = date_and_time_element.text.strip()
else:
    date_and_time=np.nan
    
if location_element:
    location = location_element.text.strip()
else:
    location=np.nan
    
artists = ', '.join([
    artist_title.text.strip()
    if (artist_title := lineup_line.find('a', class_='EventDetailsLineup__ArtistTitle-sc-e62c151a-10 jqPGJA'))
    else lineup_line.find('div', class_='EventDetailsLineup__SectionTitle-sc-e62c151a-11 drsPyw').text.strip()
    for lineup_line in soup.find_all('div', class_='EventDetailsLineup__LineupLine-sc-e62c151a-8 dvIIgU')
])
    
    
event_info = (date_and_time, location,artists)
print(event_info)

('ven. 5 janv., 19:00', '9 Rue Biscornet, 75012 Paris, France', 'Live Tribute, Live Tribute 2')


### Scraping des infos de tous les events

In [10]:
import time

In [19]:
df_dice_final = df_dice.copy()

df_dice_final[['date_and_time', 'location', 'artists']]=''

# Initialize the webdriver (in this example, using Chrome)
driver = Chrome() #webdriver.Chrome()

nb_rows_df_dice_final=df_dice_final.shape[0]

for i in range(nb_rows_df_dice_final):

    # Navigate to your website
    website_url = df_dice_final['link'].iloc[i]
    driver.get(website_url)
    
    page_source = driver.page_source
    soup = BeautifulSoup(page_source,'html.parser')
    
    date_and_time_element = soup.find('div', class_='EventDetailsTitle__Date-sc-dfc77feb-2 bITArK')
    location_element = soup.find('div', class_='EventDetailsVenue__Address-sc-456e9ee2-5 bGzzGO')



    if date_and_time_element:
        date_and_time = date_and_time_element.text.strip()
    else:
        date_and_time=np.nan

    if location_element:
        location_find = location_element.find('span')
        
        if location_find:
            location = location_find.text.strip()
        else:
            location=np.nan
    else:
        location=np.nan

    artists = ', '.join([
        artist_title.text.strip()
        if (artist_title := lineup_line.find('a', class_='EventDetailsLineup__ArtistTitle-sc-e62c151a-10 jqPGJA'))
        else lineup_line.find('div', class_='EventDetailsLineup__SectionTitle-sc-e62c151a-11 drsPyw').text.strip()
        for lineup_line in soup.find_all('div', class_='EventDetailsLineup__LineupLine-sc-e62c151a-8 dvIIgU')
    ])

    df_dice_final['date_and_time'].iloc[i] = date_and_time
    df_dice_final['location'].iloc[i] = location
    df_dice_final['artists'].iloc[i] = artists
    
    print(f"\rEvents scraped: {i+1}/{nb_rows_df_dice_final}", end="")

Events scraped: 1754/1754

In [20]:
df_dice_final.head(10)

,title,date,venue,price,link,date_and_time,location,artists
0,Les Clopes + Scimia + Blank\\,jeu. 4 janv.,Supersonic,Gratuit,https://dice.fm/event/qp2ny-les-clopes-scimia-...,NaN,NaN,
1,Plastic Palms + Clamn Dever + Panty Panty,jeu. 4 janv.,L'international,8 €,https://dice.fm/event/xym5a-plastic-palms-clam...,"jeu. 4 janv., 19:00","5 Rue Moret, 75011 Paris, France","Plastic Palms, Clamn Dever, PanTy PanTy"
2,Simone's Carpet + Lavomatic,ven. 5 janv.,Supersonic,Gratuit,https://dice.fm/event/373kp-simones-carpet-lav...,"ven. 5 janv., 19:00","9 Rue Biscornet, 75012 Paris, France","Lavomatic, Simone's Carpet"
3,The Wolf You Feed + Gargantura + Cerbera,ven. 5 janv.,L'international,7 €,https://dice.fm/event/v2wkl-the-wolf-you-feed-...,"ven. 5 janv., 19:00","5 Rue Moret, 75011 Paris, France","The Wolf You Feed, Cerbera, Gargantura"
4,La nuit des chansons tristes,ven. 5 janv.,Anti Club du Cirque électrique,8 €,https://dice.fm/event/v2vvg-la-nuit-des-chanso...,"ven. 5 janv., 19:30","Place Du Maquis Du Vercors, 75020 Paris, France","Nutsz, jakeVII, sang9"
5,Laurie Xhaard + Thomas Baignères,ven. 5 janv.,POPUP!,Gratuit,https://dice.fm/event/don3q-laurie-xhaard-thom...,"ven. 5 janv., 20:00","14 Rue Abel, 75012 Paris, France",Thomas Baignères
6,"33 Dogs, I Scream, Luciano & The Clouds, DJ Set",ven. 5 janv.,Le Sub Pigalle,"11,80 €",https://dice.fm/event/mglev-33-dogs-i-scream-l...,"ven. 5 janv., 21:00","3 Place De Clichy, 75008 Paris, France","33 Dogs, I scream, Luciano & The Clouds"
7,Gin Sonic,ven. 5 janv.,POPUP!,Gratuit,https://dice.fm/event/7wmp9-gin-sonic-5th-jan-...,"ven. 5 janv., 23:00","14 Rue Abel, 75012 Paris, France",
8,Caribbean groove dur les toits de paris,ven. 5 janv.,Tout Le Monde en Parle,À partir de Gratuit,https://dice.fm/event/lkoql-caribbean-groove-d...,"ven. 5 janv., 23:00","4 Rue Du Départ, 75015 Paris, France","Fermeture, djsprinter"
9,Pam Risourié + Fishtalk + Scarabs,sam. 6 janv.,Supersonic,Gratuit,https://dice.fm/event/56re8-pam-risouri-fishta...,"sam. 6 janv., 19:00","9 Rue Biscornet, 75012 Paris, France","Scarabs, Fishtalk, Pam Risourié"


In [21]:
# df_dice_final.to_csv('df_dice_final-WebScraping_S9.csv', index=False)

In [23]:
df_dice_final.shape

(1754, 8)

## Part 3: Quick data inspection

In [22]:
df_dice_final.isnull().sum()

title            0
date             0
venue            0
price            5
link             0
date_and_time    1
location         1
artists          0
dtype: int64

In [26]:
df_dice_final[df_dice_final['artists']==''].shape

(469, 8)